In [1]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt  
import json 
import requests
import overpy
from tqdm import tqdm
from multiprocessing.dummy import Pool
import seaborn as sns
import re
from matplotlib.pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from __future__ import print_function
import sys
import datetime
import scipy as sp
# import more packages
#!pip install ax-platform
#https://ax.dev/docs/installation.html
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Primera base New_Usuarios
New_Usuarios = pd.read_csv("New_Usuarios.csv", index_col=0, sep=';', encoding='latin-1')
New_Usuarios.reset_index(inplace=True)
New_Usuarios.head()

Revisamos el tipo de las columnas y mirar si hay datos nulos

In [ ]:
New_Usuarios.shape

In [ ]:
# Segunda base 
New_HistConsultas = pd.read_csv("New_HistConsultas.csv", index_col=0, sep=';', encoding='latin-1')
New_HistConsultas.head()

In [ ]:
New_HistConsultas.set_index('ID_Cuenta',inplace=True)
New_HistConsultas.head()

No hay datos nulos. Hay que cambiar la fecha a tipo datetime

In [ ]:
New_HistConsultas.shape

# Pregunta 1: (20%) Conocer el tipo de cliente que se comunica a la línea. Realice un análisis descriptivo detallado de las variables, tenga en cuenta realizar análisis de correlación y pruebas de hipótesis si considera necesario

In [ ]:
New_Usuarios.info()

La base de datos New_Usuarios.csv presenta 21 variables, la mayoria son categóricas. Hay variables importantes que corresponden a características demográficas de la persona.

In [ ]:
New_HistConsultas.info()

La base de datos New_HistConsultas presenta 2 variables, correspondientes a la fecha de la consulta y el tipo de consulta realizada

In [ ]:
New_Usuarios[New_Usuarios.index.duplicated(keep=False)].index.value_counts()

Hay identificadores duplicados, eso quiere decir que probablemente hay un identificador por ciudad o bien es la misma persona.

In [ ]:
New_Usuarios.describe()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

Los usuarios nuevos presentan una variación de edad entre 18 y 79 años, con una desviación estándar aproximada de 18 años. el promedio es de 48 años. Se tiene que el monto promedio que se adeuda es de \$19.900. Se observan distribuciones sesgadas a la derecha en las variables cuantitativas. En el gráfico de dispersión no se observa un patron específico.

In [ ]:
New_Usuarios.columns[[10,11,14,15]] # Variable cuantitativas

In [33]:
# Arreglar algunos errores en las variables categóricas
New_Usuarios['Tipo_persona'] = New_Usuarios['Tipo_persona'].replace(['/casado.'],'casado')
New_Usuarios['Tipo_persona'] = New_Usuarios['Tipo_persona'].replace(['casado.'],'casado')

In [ ]:
New_Usuarios.loc[(New_Usuarios['Tipo_persona']=='casado')]

In [ ]:
New_Usuarios['Tipo_persona'].value_counts()
# Hay mayor cantida de solteros. Más adelante se describirán las variables.

In [ ]:
New_Usuarios['Departamento'] = New_Usuarios['Departamento'].str.title()
New_Usuarios['Departamento'].value_counts()

In [ ]:
New_Usuarios['Departamento'].value_counts().sort_index()

In [38]:
New_Usuarios.loc[(New_Usuarios['Estrato']=='2-3')] = np.nan

In [ ]:
New_Usuarios['Estrato'].value_counts()

In [40]:
New_Usuarios['Estrato'] = pd.to_numeric(New_Usuarios['Estrato'])

In [ ]:
pal = {1.0:"brown", 2.0:"red", 3.0: "orange", 4.0: "green", 5.0: "purple", 6.0: "blue"}
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Estrato', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Estrato', palette = pal, height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
pal = {'soltero':"brown", 'unión libre':"red", 'casado': "orange", 'no sabe': "green", 'no dijo': "purple", 'soltero-casado.': "blue",'carlos': "pink"}
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Tipo_persona', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Tipo_persona', palette = pal, height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [437]:
#New_Usuarios['Estrato'] = New_Usuarios['Estrato'].astype('object',copy=False)
#New_Usuarios['y'] = New_Usuarios['y'].astype('object',copy=False)

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Departamento', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Departamento', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Tiene_plan_avanzado', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Tiene_plan_avanzado', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Tipo_Plan', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Tipo_Plan', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Es_moroso', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Es_moroso', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Tiene_plus', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Tiene_plus', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Ha_caido_mora', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Ha_caido_mora', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Correo', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Correo', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'usa_app', 'Duracion_llamada', 'Tiempo_en_espera']], 
                 plot_kws = {"color": "darkblue"}, hue = 'usa_app', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Forma_pago', 'Duracion_llamada', 'Tiempo_en_espera']], 
                 plot_kws = {"color": "darkblue"}, hue = 'Forma_pago', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Motivo_llamada', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Motivo_llamada', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Transferencia_llamada', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Transferencia_llamada', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'primera_llamada', 'Duracion_llamada', 'Tiempo_en_espera']],
                 plot_kws = {"color": "darkblue"}, hue = 'primera_llamada', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Antiguedad', 'Duracion_llamada', 'Tiempo_en_espera']], 
                               plot_kws = {"color": "darkblue"}, hue = 'Antiguedad', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'Recomienda_marca', 'Duracion_llamada', 'Tiempo_en_espera']],
                 plot_kws = {"color": "darkblue"}, hue = 'Recomienda_marca', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.pairplot(New_Usuarios[['Monto_adeudado', 'Edad', 'y', 'Duracion_llamada', 'Tiempo_en_espera']],plot_kws = {"color": "darkblue"}, 
                 hue = 'y', height=5, diag_kws = {"color":"darkblue"})
g.fig.suptitle("Relación entre pares de variables", fontsize = 26, fontweight = "bold")
plt.subplots_adjust(top=0.9)
plt.show()

In [ ]:
g = sns.catplot(x = "Estrato", kind = "count", data = New_Usuarios)
g.fig.suptitle("Número de personas por estrato", fontsize = 16, fontweight = "bold")
for ax in g.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=9)
plt.subplots_adjust(top=0.85)
g.fig.text(1,-.02, "Elaboración:", fontsize = 10, fontweight = "bold", ha = "right")
g.fig.text(1,-.08, "Saenz (2021)", fontsize = 8, ha = "right")
plt.show()

Hay mayor cantidad de personas en estrato 4. Esta variable presenta una distribución aproximadamente homogénea. 

#### Análisis de variables no numéricas

In [59]:
def Grafica(i,j,k):
    j="{}".format(j)
    Grafica=i[j].value_counts()
    print(Grafica)
    a=len(Grafica)
    plt.figure(figsize=(10,0.5*a))
    Grafica.plot.barh()
    plt.title(k)
    return Grafica

In [60]:
def Graphplot(Grafica,x_name,y_name,k):
    x,y=[],[]
    for i in range(len(Grafica)):
        a,b=Grafica.index[i],Grafica[i]
        x.append(a)
        y.append(b)
    a=len(Grafica)
    fig, ax = plt.subplots(figsize=(10,0.5*a))  
    width = 0.01 # the width of the bars 
    ind = np.arange(len(y))  # the x locations for the groups
    ax.barh(ind, y, color="steelblue",height=0.5)
    ax.set_yticks(ind+width/2)
    ax.set_yticklabels(x, minor=False)
    ax.set_xlabel(x_name)
    ax.set_ylabel(y_name)
    plt.title('title')  
    for i, v in enumerate(y):
        ax.text(v + 0.1, i , str(v), color='black', fontweight='bold')
    plt.title(k)
    plt.show()
    #plt.savefig(os.path.join('test.png'), dpi=300, format='png', bbox_inches='tight') # use format='svg' or 'pdf' for vectorial pictures
    return Grafica

In [61]:
def GraficaNueva(i,j,x_name,y_name,title):
    j="{}".format(j)
    Grafica=i[j].value_counts()
    Graphplot(Grafica,x_name,y_name,title)
    return Grafica

In [62]:
def GraficaPie2(archivo1,titulo,o):
    fig, ax = plt.subplots(figsize=(6, 8), subplot_kw=dict(aspect="equal"))
    NOMBRES,NUMERO,explode,porcentage=[],[],[],[]
    TOTAL=archivo1.sum()
    for i in range(len(archivo1)):
        p=round((archivo1[i]/TOTAL)*100,2)
        D=archivo1.index[i]+":  {}%".format(p)
        NOMBRES.append(D)
        NUMERO.append(archivo1[i])
        porcentage.append(p)
    wedges, texts = ax.pie(NUMERO, wedgeprops=dict(width=0.5), startangle=-40)

    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(arrowprops=dict(arrowstyle="-"), bbox=bbox_props, zorder=0, va="center")
    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle})
        ax.annotate(NOMBRES[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.1*y),horizontalalignment=horizontalalignment, **kw)
    ax.set_title(titulo)
    plt.show()
    return porcentage

In [ ]:
graficas = list()
for i in New_Usuarios.select_dtypes(include = ['object']).columns[1:16]:
    var2 = i
    graficas.append(Grafica(New_Usuarios,var2, var2))

El 74,16% de los nuevos usuarios presentan un estado civil de soltero, seguido está Unión libre con el 17,16%. Los registros provienen en su mayoría de Santafé de Bogotá, aproximadamente el 72,33%. Seguido está Antioquia con el 6,79% de los registros. El 90% no tiene plan avanzado, mientras que los que si tienen plan, el 27% tiene plan a seguido de plan d con el 15,57%. El 80% no son deudores morosos. De de los usuarios registrados, el 63% han caido en mora. Se observa la inserción de las nuevas tecnologías como forma de pago ya que el 60% usa pago online. Como se mencionaba anteriormente, el estrato presenta una distribución aproximadamente homogenéa. El 45% de los usuarios presentan antiguedad $legend$. Por último, el 45% de los nuevos usuarios si recomiendan la marca.

In [ ]:
for i in range(0,16):
    var2 = i
    GraficaPie2(graficas[var2],New_Usuarios.select_dtypes(include = ['object']).columns[var2+1]," ")

#### Base New_HistConsultas

Todas las variables son categóricas. Hacemos un conteo de las consultas hechas

In [ ]:
id_counts = New_HistConsultas.index.value_counts()
id_counts

Obtenemos los conteos de cada ID para ver las veces que llamó. Hay clientes que llamaron muchas veces en el transcursos de los 4 meses.

In [ ]:
tipo_consulta = New_HistConsultas['Tipo_consulta'].value_counts()
fig, ax = plt.subplots(figsize=(10, 10))
ax.bar(tipo_consulta.keys(), tipo_consulta)
for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(f'{int(height)}', xy=(rect.get_x()+rect.get_width()/2, height), 
                xytext=(0, 5), textcoords='offset points', ha='center', va='bottom')
plt.xticks(rotation = 45)
plt.show

In [ ]:
Tipos_llamadas = Grafica(New_HistConsultas,"Tipo_consulta", "Numero de tipos de consulta")

En la base de datos New_HistConsultas se enuncian 5 tipos de consulta, cuya consulta más frecuente son Procesos web con el 38,15% de los registros, es decir 489.480. Seguidamente se encuentra agendamiento citas con el 26,25%. La consulta menos frecente en las llamadas es información general, con apenas el 2,8% del total, es decir 35.828 consultas.

In [ ]:
GraficaPie2(Tipos_llamadas,"Tipos de Consulta"," ")

In [ ]:
New_HistConsultas['Fecha_consulta'] = pd.to_datetime(New_HistConsultas['Fecha_consulta'], format = '%Y-%m-%d %H:%M:%S')
print(New_HistConsultas.info())

In [ ]:
rcParams['figure.figsize'] = 15, 6
New_HistConsultas['Fecha_consulta_dia']=pd.to_datetime(New_HistConsultas['Fecha_consulta'],format="%Y-%m-%d").dt.dayofweek
New_HistConsultas

In [ ]:
########################## Llamadas por el día de la semana 
#check datatype of index
#convert to time series:
Llamadas = New_HistConsultas["Fecha_consulta_dia"].value_counts()
Llamadas = Llamadas.sort_index()
#Llamadas.index = pd.to_datetime(Llamadas.index)
Llamadas = Llamadas.to_frame(name="LLamadas")
####Graficar la Serie#####
Llamadas.rename(index={"0": 'Lunes'})
Llamadas.index = ['Lunes', 'Martes', 'Miercoles',"Jueves","Viernes","Sabado"]
plt.plot(Llamadas)
plt.title('Dias de la semana') 
plt.show()

In [ ]:
Llamadas

El día de la semana en el que igresan más llamadas es el martes, seguido del miércoles. Se observa además en la gráficas de arriba que la frecuencia de las llamadas va disminuyendo a medida que se acerca el fin de semana. El sábado es el día en donde menos llamadas se presentan. 

In [ ]:
rcParams['figure.figsize'] = 15, 6
New_HistConsultas['Fecha_consulta_dia2']=pd.to_datetime(New_HistConsultas['Fecha_consulta'],format="%Y-%m-%d").dt.date
New_HistConsultas

In [ ]:
Llamadas2=New_HistConsultas['Fecha_consulta_dia2'].value_counts()
Llamadas2=Llamadas2.sort_index()
#Llamadas.index = pd.to_datetime(Llamadas.index)
Llamadas2=Llamadas2.to_frame(name="LLamadas")
####Graficar la Serie#####
Llamadas2.rename(index={"0": 'Lunes'})
plt.plot(Llamadas2)
plt.title('Serie diaria') 
plt.show()

In [ ]:
Llamadas2

Hay que mencionar que la base de datos New_HistConsultas presenta una variable de fecha de consulta, con un horizonte temporal de 4 meses, compredido entre el 1 de junio de 2015 y el 29 de septiembre de 2015. En el gráfico de arriba se observa que la serie presenta un ciclo bastante marcado, debido principalmente a que las llamadas disminuyen bastante los viernes y sábado. La mayor cantidad de consultas realizadas por los usuarios de da durante el més de agosto y las primeras dos semanas de septiembre. donde superan el registro de 20.000 llamadas diarias. 

In [ ]:
rcParams['figure.figsize'] = 15, 6
New_HistConsultas['Fecha_consulta_mes']=pd.to_datetime(New_HistConsultas['Fecha_consulta'],format="%Y-%m-%d").dt.month
New_HistConsultas

In [ ]:
Llamadas3=New_HistConsultas['Fecha_consulta_mes'].value_counts()
Llamadas3=Llamadas3.sort_index()
#Llamadas.index = pd.to_datetime(Llamadas.index)
Llamadas3=Llamadas3.to_frame(name="LLamadas")
####Graficar la Serie#####
Llamadas3.rename(index={"6": 'junio'})
Llamadas3.index = ['junio', 'julio', 'agosto','septiembre']
plt.plot(Llamadas3)
plt.title('Serie diaria') 
plt.show()

In [ ]:
Llamadas3

Por último se observa que el mes de agosto presentó mayor cantidad de llamadas, aproximadamente 380.400 llamadas. Seguido está el mes de septiembre. El mes de junio presenta menos frecuencia de llamadas. 

Separamos conteos por tipo de consulta

In [ ]:
New_HistConsultas = pd.read_csv("New_HistConsultas.csv", index_col=0, sep=';', encoding='latin-1')
New_HistConsultas['Fecha_consulta'] = pd.to_datetime(New_HistConsultas['Fecha_consulta'], format = '%Y-%m-%d %H:%M:%S')
consultas_id = pd.pivot_table(New_HistConsultas, index='ID_Cuenta', columns='Tipo_consulta', aggfunc='count', fill_value=0).droplevel(0,axis=1)
consultas_id

#### Correlación

La asociación lineal entre las variables cuantitativas es baja. Se filtró por: 
Estado civil de la persona, departamento == Santafé de Bogotá, si es morosa, si alguna vez ha caido en mora, estrato == 1, estrato == 6, Antiguedad == legend, y $y == 1$. No se halló asociación lineal significativa. Se debe observar la asociación no lineal.

In [ ]:
cuantitativas = New_Usuarios[['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']]
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(cuantitativas.corr(), vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas con persona soltera", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(New_Usuarios[New_Usuarios['Tipo_persona']=='soltero'][['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']].corr(), 
            vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas con Bogotá", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(New_Usuarios[New_Usuarios['Departamento']=='Santafé De Bogotá'][['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']].corr(), 
            vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas con persona morosa", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(New_Usuarios[New_Usuarios['Es_moroso']=='si'][['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']].corr(), 
            vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas con persona que ha caido en mora", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(New_Usuarios[New_Usuarios['Ha_caido_mora']=='si'][['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']].corr(), 
            vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas con persona de estrato 1", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(New_Usuarios[New_Usuarios['Estrato']==1][['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']].corr(), 
            vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas con persona de estrato 6", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(New_Usuarios[New_Usuarios['Estrato']==6][['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']].corr(), 
            vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas con persona antigua", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(New_Usuarios[New_Usuarios['Antiguedad']=='Legend'][['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']].corr(), 
            vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
fig.suptitle("Correlación simple entre variables cuantitativas con y = 1", fontsize = 16, fontweight = "bold", x = 0.43)
sns.heatmap(New_Usuarios[New_Usuarios['y']==1][['Monto_adeudado', 'Edad', 'Duracion_llamada', 'Tiempo_en_espera']].corr(), 
            vmin = -1, vmax = 1, ax = ax, cmap = "RdYlGn", annot = True)
plt.subplots_adjust(top = 0.85)
plt.show()

Agregamos los datos de conteo conseguidos en el dataset anterior

In [ ]:
New_Usuarios = pd.read_csv("New_Usuarios.csv", index_col=0, sep=';', encoding='latin-1')
usuarios_df_full = pd.merge(consultas_id, New_Usuarios,left_index=True, right_index=True)
usuarios_df_full.head()

In [ ]:
usuarios_df_full.info()

Estandarizamos los strings y llenamos valores faltantes con la moda

In [119]:
for column in usuarios_df_full:
    if usuarios_df_full[column].dtype== 'object':
        usuarios_df_full[column]=usuarios_df_full[column].str.lower().replace('[#,@,&,/]', '')
        usuarios_df_full[column].fillna(usuarios_df_full[column].mode()[0], inplace=True)

Revisamos si faltan datos

In [ ]:
usuarios_df_full.isna().sum()

FAMD para el punto 1, para ver variables importantes, ahí mismo se hacen correlaciones. Imputar con moda para datos nulos.


Para los perfiles de clientes se HCPC(Hierarchical clustering for principal components)

In [122]:
usuarios_df_full.to_csv('full_dataset.csv')

# Pregunta 2: (20%) Implementando modelos no supervisados identifica un perfil de cliente basado en las características de las personas que se comunican a la línea. Y genera una propuesta personalizada de atención para cada perfil encontrado.

Se encuentra en el jupyter notebook anexo: $\pmb{Prueba\_Emtelco\_Juan\_Saenz\_punto2.ipynb}$

# Pregunta 3: (60%) Utilizando modelos supervisados, prediga la variable ‘y’ que se encuentra en la base de datos, tenga en cuenta lo siguiente:

## 3.1 Use al menos uno de los siguientes modelos: Regresión lineal, Regresión logística, Regresión ridge, Regresión lasso o elastic-net. Y explique el porqué de su selección e interprete. (5%)

In [ ]:
usuarios_df = pd.read_csv('full_dataset.csv', index_col=0)
print(usuarios_df.shape)

Por lo anterior de FAMD (Factor analysis of mixed data - análisis factorial de datos mixtos), no tomamos todas las variables a trabajar.

In [4]:
variables=['Actualización datos', 'Agendamiento citas','Información general',	
            'Peticiones y quejas', 'Procesos web', 'Tipo_persona', 'Departamento',
            'Tipo_Plan', 'Tipo_persona', 'Estrato','Antiguedad', 'Motivo_llamada',
            'Forma_pago', 'y']
usuarios_df=usuarios_df[variables]

Es necesario pasar los datos categóricos a one hot encoding

In [ ]:
usuarios_df_dummies=pd.get_dummies(usuarios_df,
                                  columns=[x for x in usuarios_df.columns if x not in usuarios_df.describe().columns])
usuarios_df_dummies.head()

Se procede a diseñar modelos de aprendizaje automático. En el primer caso, $\pmb{\text{se usa regresión logística, ya que la variable respuesta es una variable binaria.}}$ 

Revisamos la cantidad de datos de $y$ para ver si están desbalanceados

In [ ]:
usuarios_df['y'].value_counts()/usuarios_df.shape[0]

Hay un ligero desbalanceo, se proseguirá sin hacer cambios por ahora.

Se separan los datos en entrenamiento y prueba, y se hace un escalamiento de datos para evitar sesgo en las variables numéricas

In [6]:
X_train, X_test, y_train, y_test = train_test_split(usuarios_df_dummies.drop('y', axis=1), usuarios_df_dummies['y'],
                                                   test_size=0.3, random_state=13)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape + X_test_scaled.shape)

#### Regresión logística

In [ ]:
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train_scaled, y_train)

Se procede a hacer predicciones, y observar las capacidades de predicción

In [156]:
pred_logisticRegr=logisticRegr.predict(X_test_scaled)

In [ ]:
score_lr = logisticRegr.score(X_test_scaled, y_test)
print(score_lr)

El modelo casi siempre predice el caso "0", generando constantemente falsos positivos

## 3.2 Use al menos 3 modelos de machine learning para predecir la variable objetivo, siéntase libre de implementar los modelos que considere necesario (no usar los modelos sugeridos en el punto 3.1). (35%)

#### Árbol de decisión

Se propone luego el algoritmo de arbol de decisión. Este se espera que funcione correctamente debido a que es no parmétrico

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train_scaled, y_train)

In [ ]:
score_treecl=tree.score(X_test_scaled, y_test)
print(score_treecl)

In [ ]:
pred_treecl=tree.predict(X_test_scaled)
cm = metrics.confusion_matrix(y_test, pred_treecl)
print(cm)

In [171]:
tree = DecisionTreeClassifier()

gs = GridSearchCV(tree,
                  param_grid = {'max_depth': range(1, 10)},
                  cv=10,
                  n_jobs=5,
                  scoring='accuracy')
cv_tree1=gs.fit(X_train_scaled, y_train)

In [ ]:
gs.best_estimator_

In [ ]:
score_treecl=gs.score(X_test_scaled, y_test)
print(score_treecl) # Mejoró el modelo con la grilla de parámetros que se buscó

#### AdaBoostClassifier

Este tiene una precisión menor a la regresión logística, pero sus predicciones son más variadas.
Se propone ahora aplicar boosting, posiblemente mejorando el caso anterior

In [ ]:
adb = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),
                         n_estimators=10,learning_rate=0.3)
adb.fit(X_train_scaled, y_train)

In [ ]:
score_ABcl=adb.score(X_test_scaled, y_test)
print(score_ABcl)

In [ ]:
pred_ABcl=adb.predict(X_test_scaled)
cm = metrics.confusion_matrix(y_test, pred_ABcl)
print(cm)

#### Random Forest

De nuevo se tienen problemas conlos falsos positivos, aunque menos que en regresión logística.

Se proponen ahora random forests.

In [ ]:
rf = RandomForestClassifier(n_estimators=5, max_depth=10)
rf.fit(X_train_scaled, y_train)

In [ ]:
score_rf=rf.score(X_test_scaled, y_test)
print(score_rf)

In [ ]:
pred_rf=rf.predict(X_test_scaled)

In [202]:
n_estimators =  [1,2,3,4,5,6,7,8,9,10]
max_depth = [1,2,3,4,5,6,7,8,9,10]
min_samples_split = [1, 2, 3]
min_samples_leaf = [1, 2, 3]
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }

In [ ]:
rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = 5, verbose = 2)
rf_cv=grid_search.fit(X_train_scaled, y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
score_rf=grid_search.score(X_test_scaled, y_test)
print(score_rf) # Mejoró el modelo con la grilla de parámetros que se buscó

Incluso si con el modelo de Random Forest perdemos gran parte de la interpretación del modelo que nos dio un árbol de decisión simple, aún podemos obtener la importancia de cada característica.

In [ ]:
importances1 =rf_cv.best_estimator_.feature_importances_
importances_df1=pd.DataFrame({'importances':importances1,'feauture':usuarios_df_dummies.drop('y', axis=1).columns})
importances_df1=importances_df1.sort_values(by=['importances'],ascending=False)
importances_df1

In [ ]:
fig = plt.figure(figsize = (10, 5))
plt.bar(importances_df1.feauture[:15],importances_df1.importances[:15], color ='maroon',
        width = 0.4)
plt.xticks( rotation='vertical')

Las variables más importantes para la predicción son: Peticiones y quejas, Procesos web, Motivo_llamada_m20, Motivo_llamada_m17 e Información general. También entre las 15 más importantes se encuentran variables como la antiguedad, que la persona sea de estrato 2, la persona con estado civil de unión libre. 

De nuevo tenemos problemas con la cantidad de falsos positivos

Se postula que el problema principal yace en los datos. Principalmente su cantidad de variables categóricas. A un menor nivel, es posible que el desbalanceo de los datos esté causando parte de estos inconvenientes. Si se quiere continuar usando estos métodos se recomienda arreglar estos asuntos. 

En su lugar se propone el uso de una red neuronal

## 3.3 Defina una arquitectura de red neuronal e impleméntela para predecir la variable respuesta. (20%)

In [ ]:
print(X_train.shape)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),

                optimizer=tf.keras.optimizers.Adam(), 

                metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
model.evaluate(X_test_scaled, y_test)

In [211]:
pred_NN=np.round(model.predict(X_test_scaled))

Ahora se va a optimizar la red y escoger los mejores parámetros dentro de una grilla:

In [214]:
X_train_full, X_test, y_train_full, y_test = train_test_split(usuarios_df_dummies.drop('y', axis=1), 
                                                              usuarios_df_dummies['y'],
                                                              test_size=0.2, random_state=13)

In [ ]:
X_train_full.shape, X_test.shape, y_train_full.shape, y_test.shape 

In [216]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full,y_train_full, test_size=0.2, random_state=13)

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
215712 + 53928 + 67411

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
print(X_train.shape + X_test.shape + X_val.shape)

In [220]:
##Busqueda de los hiperparámetros
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# This returns a multi-layer-perceptron model in Keras.
def get_keras_model(num_hidden_layers, 
                    num_neurons_per_layer, 
                    dropout_rate, 
                    activation):
    # create the MLP model.
    
    # define the layers.
    inputs = tf.keras.Input(shape=(X_train.shape[1],))  # input layer.
    x = layers.Dropout(dropout_rate)(inputs) # dropout on the weights.
    
    # Add the hidden layers.
    for i in range(num_hidden_layers):
        x = layers.Dense(num_neurons_per_layer, 
                         activation=activation)(x)
        x = layers.Dropout(dropout_rate)(x)
    
    # output layer.
    outputs = layers.Dense(1, activation='sigmoid')(x) #Binary classification
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [221]:
# This function takes in the hyperparameters and returns a score (Cross validation).
def keras_mlp_cv_score(parameterization, weight=None):
    
    model = get_keras_model(parameterization.get('num_hidden_layers'),
                            parameterization.get('neurons_per_layer'),
                            parameterization.get('dropout_rate'),
                            parameterization.get('activation'))
    
    opt = parameterization.get('optimizer')
    opt = opt.lower()
    
    learning_rate = parameterization.get('learning_rate')
    
    if opt == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif opt == 'rms':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    
    NUM_EPOCHS = 5
    
    # Specify the training configuration.
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=['mse'])

    data = X_train
    labels = y_train
    
    # fit the model using validation_data=(X_val,y_val) validation set.
    res = model.fit(data, labels, epochs=NUM_EPOCHS, batch_size=parameterization.get('batch_size'),
                    validation_data=(X_val,y_val))
    
    # look at the last 10 epochs. Get the mean and standard deviation of the validation score.
    last10_scores = np.array(res.history['val_loss'][-10:])
    mean = last10_scores.mean()
    sem = last10_scores.std()
    
    # If the model didn't converge then set a high loss.
    if np.isnan(mean):
        return 9999.0, 0.0
    
    return mean, sem

In [222]:
# Define the search space.
parameters=[
    {
        "name": "learning_rate",
        "type": "range",
        "bounds": [0.001, 0.002],
        "log_scale": True,
    },
    {
        "name": "dropout_rate",
        "type": "range",
        "bounds": [0.01, 0.02],
        "log_scale": True,
    },
    {
        "name": "num_hidden_layers",
        "type": "range",
        "bounds": [1, 2],
        "value_type": "int"
    },
    {
        "name": "neurons_per_layer",
        "type": "range",
        "bounds": [16, 32],
        "value_type": "int"
    },
    {
        "name": "batch_size",
        "type": "choice",
        "values": [16, 32],
    },
    
    {
        "name": "activation",
        "type": "choice",
        "values": ['tanh', 'sigmoid', 'relu'],
    },
    {
        "name": "optimizer",
        "type": "choice",
        "values": ['adam', 'rms'],
    },
]

In [ ]:
init_notebook_plotting()

ax_client = AxClient()

# create the experiment.
ax_client.create_experiment(
    name="keras_experiment",
    parameters=parameters,
    objective_name='keras_cv',
    minimize=True)

def evaluate(parameters):
    return {"keras_cv": keras_mlp_cv_score(parameters)}

In [ ]:
for i in range(10):
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

In [ ]:
ax_client.get_trials_data_frame().sort_values('trial_index')

In [ ]:
best_parameters, values = ax_client.get_best_parameters()

# the best set of parameters.
for k in best_parameters.items():
  print(k)

print()

# the best score achieved.
means, covariances = values
print(means)

## 3.4 Optimice los modelos anteriormente planteados en el punto 3.2 y 3.3 (20%).

Para el Árbol de decisión y el Random Forest se formuló una grilla de parámetros para encontrar el modelo que mejor score arrojara. En la red neuronal se hizo lo mismo, con una grilla de parámetros que incluía funciones de activación, número de capas ocultas, neuronas por capa, batch size y optimizadores. 

Adicionalmente en este punto se va a realizar la estimación de los modelos de machine learning y la red neuronal anteriormente mencionados, teniendo en cuenta la mejor en el balanceo de los datos.

Con estas demostraciones se busca entonces mejorar el balanceo de los datos

In [ ]:
case_0=usuarios_df_dummies[usuarios_df_dummies['y']==0]
case_1=usuarios_df_dummies[usuarios_df_dummies['y']==1]

case_1_upsample = resample(case_1,
             replace=True,
             n_samples=len(case_0),
             random_state=42)
print(len(case_1_upsample), len(case_0))

In [ ]:
usuarios_df_upsample=pd.concat([case_0, case_1_upsample])
print(usuarios_df_upsample.shape)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(usuarios_df_upsample.drop('y', axis=1), 
                                                    usuarios_df_upsample['y'],
                                                    test_size=0.3, random_state=13)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape, X_test_scaled.shape)

Repetimos regresión logística

In [ ]:
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train_scaled, y_train)

Se procede a hacer predicciones, y observar las capacidades de predicción

In [13]:
pred_logisticRegr=logisticRegr.predict(X_test_scaled)

In [ ]:
score_lr = logisticRegr.score(X_test_scaled, y_test)
print(score_lr)

Para observar más a fondo cómo trabaja la predicción de los datos, se usa una matriz de confusión.

In [ ]:
cm = metrics.confusion_matrix(y_test, pred_logisticRegr)
print(cm)

Se redujo la precisión, pero las predicciones no son sesgadas

Repetimos arboles de decisión

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train_scaled, y_train)

In [ ]:
score_treecl=tree.score(X_test_scaled, y_test)
print(score_treecl)

In [ ]:
pred_treecl=tree.predict(X_test_scaled)
cm = metrics.confusion_matrix(y_test, pred_treecl)
print(cm)

Se mejoró la predicción y la variedad de precisión

Se repite boosted trees

In [ ]:
adb = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),
                         n_estimators=10,learning_rate=0.3)
adb.fit(X_train_scaled, y_train)

In [ ]:
score_ABcl=adb.score(X_test_scaled, y_test)
print(score_ABcl)

In [ ]:
pred_ABcl=adb.predict(X_test_scaled)

cm = metrics.confusion_matrix(y_test, pred_ABcl)
print(cm)

In [ ]:
rf = RandomForestClassifier(n_estimators=30, max_depth=10)
rf.fit(X_train_scaled, y_train)

In [ ]:
score_rf=rf.score(X_test_scaled, y_test)
print(score_rf)

In [ ]:
pred_rf=rf.predict(X_test_scaled)

cm = metrics.confusion_matrix(y_test, pred_rf)
print(cm)

Se repite la red neuronal

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),

                optimizer=tf.keras.optimizers.Adam(), 

                metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
model.evaluate(X_test_scaled, y_test)

In [29]:
pred_NN=np.round(model.predict(X_test_scaled))

In [ ]:
cm = metrics.confusion_matrix(y_test, pred_NN)
print(cm)

En conclusión, después del arreglo de los datos, el modelo de Arboles de decisión es el mejor, debido a tanto su falta de sesgo, como su precisión. 

el problema principal cae en la base de datos siendo "muy compleja" para un sistema de clasificación binario sencillo.

Mediante las funciones keras_mlp_cv_score y get_keras_model construidas en el punto de optimización de la red neuronal, se arroja un 'learning_rate' de 0.001186383338267293 y 'dropout_rate' de 0.017273042056272076, parámetros que se usan para controlar y corregir el sobreajuste de la red. 

Adicionalmente se corrigio el desbalanceamiento de los datos y con ello disminuye el sesgo.

## 3.6 Compare los 5 modelos y seleccione el mejor, y explique cuál fue el criterio por el cual lo eligió. (10%)

In [ ]:
score_lr, score_treecl, score_ABcl, score_rf, model.evaluate(X_test_scaled, y_test)

In [ ]:
metrics.confusion_matrix(y_test, pred_logisticRegr)

In [ ]:
metrics.confusion_matrix(y_test, pred_treecl)

In [ ]:
metrics.confusion_matrix(y_test, pred_ABcl)

In [ ]:
metrics.confusion_matrix(y_test, pred_rf)

In [ ]:
metrics.confusion_matrix(y_test, pred_NN)

Bajo estos criterior que son el accuracy y la confusio_matrix, es mejor el modelo de árboles de decisión.

In [ ]:
# Accuracy = TP + TN / (TP + TN + FP + FN)
# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN)  Also known as sensitivity, or True Positive Rate
# F1 = 2 * Precision * Recall / (Precision + Recall) 
print('Accuracy_m1: {:.2f}'.format(accuracy_score(y_test, pred_logisticRegr)))
print('Accuracy_m2: {:.2f}'.format(accuracy_score(y_test, pred_treecl)))
print('Accuracy_m3: {:.2f}'.format(accuracy_score(y_test, pred_ABcl)))
print('Accuracy_m4: {:.2f}'.format(accuracy_score(y_test, pred_rf)))
print('Accuracy_m5: {:.2f}'.format(accuracy_score(y_test, pred_NN)))

In [ ]:
print('Precision_m1: {:.2f}'.format(precision_score(y_test, pred_logisticRegr, average='micro')))
print('Precision_m2: {:.2f}'.format(precision_score(y_test, pred_treecl, average='micro')))
print('Precision_m3: {:.2f}'.format(precision_score(y_test, pred_ABcl, average='micro')))
print('Precision_m4: {:.2f}'.format(precision_score(y_test, pred_rf, average='micro')))
print('Precision_m5: {:.2f}'.format(precision_score(y_test, pred_NN, average='micro')))

In [ ]:
print('Recall_m1: {:.2f}'.format(recall_score(y_test, pred_logisticRegr, average='micro')))
print('Recall_m2: {:.2f}'.format(recall_score(y_test, pred_treecl, average='micro')))
print('Recall_m3: {:.2f}'.format(recall_score(y_test, pred_ABcl, average='micro')))
print('Recall_m4: {:.2f}'.format(recall_score(y_test, pred_rf, average='micro')))
print('Recall_m5: {:.2f}'.format(recall_score(y_test, pred_NN, average='micro')))

In [ ]:
print('F1_m1: {:.2f}'.format(f1_score(y_test, pred_logisticRegr, average='micro')))
print('F1_m2: {:.2f}'.format(f1_score(y_test, pred_treecl, average='micro')))
print('F1_m3: {:.2f}'.format(f1_score(y_test, pred_ABcl, average='micro')))
print('F1_m4: {:.2f}'.format(f1_score(y_test, pred_rf, average='micro')))
print('F1_m5: {:.2f}'.format(f1_score(y_test, pred_NN, average='micro')))

In [ ]:
# Combined report with all above metrics
print(classification_report(y_test, pred_logisticRegr)) # Modelo 1

In [ ]:
print(classification_report(y_test, pred_treecl)) # Modelo 2

In [ ]:
print(classification_report(y_test, pred_ABcl)) # Modelo 3

In [ ]:
print(classification_report(y_test, pred_rf)) # Modelo 4

In [ ]:
print(classification_report(y_test, pred_NN)) # Modelo 5

En conclusión, después del arreglo de los datos, el modelo de Arboles de decisión es el mejor, debido a tanto su falta de sesgo, como su precisión. 

el problema principal cae en la base de datos siendo "muy compleja" para un sistema de clasificación binario sencillo.

## 3.7 Una vez finalizado lo anterior concluya y proponga una metodología de despliegue del mejor modelo seleccionado. (5%)